<a href="https://colab.research.google.com/github/pscspy/public/blob/main/arquivado/002/002_WebScrapping_(B3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WebScrapping FII (+B3)
* O Script realiza um WebScrapping do site FundsExplorer, selecionando informações específicas de ativos de interesse e as contatenando em uma lista. Para o exemplo, considerou-se ativos aleatórios presentes no iFIX e informações relevantes:
  * Ultimos dividendos (ultimo mês, três meses, seis e doze);
  * Patrimônio Líquido
  * Valor patrimonial por cota

* Adversamente, devido inconsistência de informações, este script coleta também da B3 e posteriormente exporta para arquivo xls.

# WebScrapping FundsExplorer


In [ ]:
#@title 1. Import
#@markdown Importando e checando (200) reposta do site.
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = requests.get('https://www.fundsexplorer.com.br/ranking')
url

<Response [200]>

In [ ]:
#@title 2. Criando DataFrame
#@markdown Table to pd.DataFrame e exibindo lista. 
#   Investigating.
url_bs = BeautifulSoup(url.text,'html.parser')
#   Selecting table.
url_table = url_bs.findAll(attrs={'id':'scroll-wrapper'})
#   Table to Bs (Indexing) - type(table)
table = url_table[0].findAll('table')
#   Table to DataFrame
fii_df = pd.read_html(str(table))[0]
#   Lista por ordem alfabética
# str(list(fii_df['Códigodo fundo'].sort_values()))

In [ ]:
#@title Selecionando colunas
#@markdown Excluindo colunas consideradas irrelevantes.
new_fii_df = fii_df.drop(fii_df.columns[[1,2,3,5,6,7,8,9,10,11,12,13,14,15,18,19,20,21,22,23,24,25]], axis=1)

In [ ]:
#@title Ajustando formatação
#@markdown Substituindo caracteres e formatação para compatibilidade.

#   R$ = [r raw] [^começa com] [\$ \string] [ <espaço>]
new_fii_df = new_fii_df.replace(to_replace=r'^R\$ ', value='',regex=True)
#   Separação de milhar (mantive vírgula para decimal)
new_fii_df['PatrimônioLíq.'] = new_fii_df['PatrimônioLíq.'].replace('.','')

In [ ]:
#@title Concluindo formatação
#@markdown Renomeando colunas.
new_fii_df.rename(columns={'Códigodo fundo':'FII', 'Dividendo': 'Ult_Dv', 'PatrimônioLíq.': 'Pat_Lq','VPA':'VPA_Rnk'}, inplace=True)


In [ ]:
#@title Listando Fii's (aleatórios do iFIX)
#@markdown Criando lista de Ativos de interesse.
fii_list = ['KNIP11','KNCR11','HGLG11','IRDM11','KNRI11','CPTS11','XPLG11','HCTR11','RECR11','MXRF11','HGRU11','BCFF11','BRCR11','HFOF11']

In [ ]:
#@title Selecionando Ativos de interesse
#@markdown Dos danos do WebScrapping selecionando apenas ativos de interesse.
fiinal_df = new_fii_df[new_fii_df['FII'].isin(fii_list)]

In [ ]:
#@title Segundo WebScrapping.
#@markdown Dos ativos de interesse individualmente recolhendo informações acerca dos últimos dividendos.

dvs_fii = pd.DataFrame()

for i in fii_list:
  zero = requests.get(f'https://www.fundsexplorer.com.br/funds/{i}')
  um = BeautifulSoup(zero.text,'html.parser')
  dois = um.findAll(attrs={'class':'table-responsive'})
  tres = dois[0].findAll('table')
  quatro = pd.read_html(str(tres))[0]
  cinco = quatro.drop(quatro.columns[5], axis=1)
  seis = cinco.drop(1)
  seis.loc[0,'Proventos'] = f'{i}'
  dvs_fii = dvs_fii.append(seis)
dvs_fii = dvs_fii.replace(to_replace=r'^R\$ ', value='',regex=True)
dvs_fii.rename(columns={'Proventos':'FII', 'Último': 'Ult_Dv_0', '3 meses': '3','6 meses':'6','12 meses':'12'}, inplace=True)

In [ ]:
#@title Consolidando tabelas
#@markdown Mantido Ult_Dv_0 e Ult_Dv apenas para checar consistência interna de informações.

var01 = dvs_fii
var02 = fiinal_df
consolidado = pd.merge(var01,var02)
consolidado

,FII,Ult_Dv_0,3,6,12,Ult_Dv,Pat_Lq,VPA_Rnk
0,KNIP11,"0,5000","1,1500","4,2700","12,9600","0,50","7.792.811.693,25","96,64"
1,KNCR11,"1,1000","3,4000","7,0500","12,8000","1,10","5.594.359.606,83","100,68"
2,HGLG11,"1,1000","3,3000","8,8000","16,0500","1,10","3.454.272.241,76","147,26"
3,IRDM11,"0,7016","2,3400","6,2191","13,7443","0,70","3.450.887.726,43","94,72"
4,KNRI11,"0,9100","2,7300","5,3900","10,3800","0,91","3.871.638.686,32","160,27"
5,CPTS11,"0,3700","2,2700","5,5700","12,1700","0,37","2.911.469.296,79","91,61"
6,XPLG11,"0,7400","2,2200","4,3600","8,3000","0,74","3.102.791.995,75","114,49"
7,HCTR11,"1,1000","3,3000","7,1500","16,6700","1,10","2.681.663.950,61","121,43"
8,RECR11,"0,7350","1,7773","4,7478","13,0406","0,74","2.520.730.032,03","95,33"
9,MXRF11,"0,0800","0,2800","0,6100","1,2000","0,08","2.305.298.861,59","10,20"


# WebScrapping B3

In [ ]:
#@title Listando Código ISIN
#@markdown Devido eventos adversos, é especificado ISIN recorrente.

isn_list = ['BRKNIPCTF001','BRKNCRCTF000','BRHGLGCTF004','BRIRDMCTF004','BRKNRICTF007','BRCPTSCTF004','BRXPLGCTF002','BRHCTRCTF003','BRRECRCTF004','BRMXRFCTF008','BRHGRUCTF002','BRBCFFCTF000','BRBRCRCTF000','BRHFOFCTF002']

In [ ]:
#@title WebScrapping B3
#Para evitar erros de requisição (bloqueio de segurança), há intervalo de 15s entre cada ativo.
import time
from IPython.display import clear_output                    # apenas para debug
dividendos = pd.DataFrame()                                 # setup and clearing df
x_len = len(fii_list)                                       # debugger loop setup 0
y_len = 0                                                   # debugger loop setup 1

for i in fii_list:
  y_temp = y_len+1                                          # debugger loop setup 2
  y_len = y_temp                                            # debugger loop setup 3
#  zero = requests.get(f'https://sistemaswebb3-listados.b3.com.br/fundsPage/main/24960430000113/{i}/7/events')
  zero = requests.get(f'https://bvmf.bmfbovespa.com.br/Fundos-Listados/FundosListadosDetalhe.aspx?Sigla={i}&tipoFundo=Imobiliario&aba=abaEventosCorporativos&idioma=pt-br')
  um = BeautifulSoup(zero.text,'html.parser')               # html0
  dois = um.findAll(attrs={'id':'divDividendo'})            # html1
  tres = dois[0].findAll('table')                           # html2
  quatro = pd.read_html(str(tres))[0]                       # pd0
  for n in isn_list:
    cinco = quatro
    xinco = quatro
    xinco = quatro.loc[quatro['Código ISIN'] ==f'{n}']
    xeis = xinco[0:12]
    cinco.loc[cinco['Código ISIN']==f'{n}', '12_B3'] = xeis.loc[xeis['Código ISIN'] ==f'{n}','Valor (R$)'].sum()
  seis = cinco.assign(FII=f'{i}')                           # formating - FII {i}
  sete = seis.drop(quatro.columns[[0,2,7]], axis=1)         # formating - columns
  dividendos = dividendos.append(sete.iloc[0])              # exporting to df       bug futuro colocar iloc na linha q tiver f'{n}' na coluna Código ISIN 
  print(f'{y_temp}/{x_len}')                                # loop count debugger 4
  time.sleep(15)                                            # just test to avoid trafic ban
  clear_output(wait=True)                                   # debugger refresh 5
dividendos.rename(columns={'Negócios com até':'DataCom', 'Valor (R$)': 'Ult_Dv_B3', 'Relativo a': 'Mês_Ref','Início de Pagamento':'Data_Pag'}, inplace=True)
dividendos['Ult_Dv_B3'] = (dividendos['Ult_Dv_B3'] / 100000000000)
dividendos['12_B3'] = (dividendos['12_B3']/100000000000)
dividendos['Ult_Dv_B3'] = dividendos['Ult_Dv_B3'].astype(str)
dividendos['12_B3'] = dividendos['12_B3'].astype(str)
dividendos['Ult_Dv_B3'] = dividendos['Ult_Dv_B3'].str.replace('.',',')
dividendos['12_B3'] = dividendos['12_B3'].str.replace('.',',')
dividendos                                                  # print finished table

<ipython-input-11-4a9a4f780664>:35: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividendos['Ult_Dv_B3'] = dividendos['Ult_Dv_B3'].str.replace('.',',')
<ipython-input-11-4a9a4f780664>:36: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  dividendos['12_B3'] = dividendos['12_B3'].str.replace('.',',')


,Código ISIN,DataCom,Ult_Dv_B3,Mês_Ref,Data_Pag,12_B3,FII
0,BRKNIPCTF001,30/11/2022,"0,5",NOVEMBRO 2022/2022,13/12/2022,"12,96",KNIP11
0,BRKNCRCTF000,30/11/2022,"1,1",NOVEMBRO 2022/2022,13/12/2022,"12,42",KNCR11
0,BRHGLGCTF004,30/11/2022,"1,1",Novembro/2022,14/12/2022,"16,05",HGLG11
0,BRIRDMCTF004,09/12/2022,"0,701563187",Novembro-2022/2022,16/12/2022,"13,744345626",IRDM11
0,BRKNRICTF007,30/11/2022,"0,91",NOVEMBRO 2022/2022,14/12/2022,"10,38",KNRI11
0,BRCPTSCTF004,12/12/2022,"0,37",Novembro-2022/2022,19/12/2022,"12,17",CPTS11
0,BRXPLGCTF002,30/11/2022,"0,74",11-2022/2022,14/12/2022,"8,3",XPLG11
0,BRHCTRCTF003,07/12/2022,"1,1",11-2022/2022,14/12/2022,"16,67",HCTR11
0,BRRECRCTF004,07/12/2022,"0,735",NOVEMBRO/2022/2022,14/12/2022,"13,0406",RECR11
0,BRMXRFCTF008,30/11/2022,"0,08",Novembro-2022/2022,14/12/2022,"1,2",MXRF11


In [ ]:
var01 = consolidado
var02 = dividendos
consolidado = pd.merge(var01,var02)
consolidado

,FII,Ult_Dv_0,3,6,12,Ult_Dv,Pat_Lq,VPA_Rnk,Código ISIN,DataCom,Ult_Dv_B3,Mês_Ref,Data_Pag,12_B3
0,KNIP11,"0,5000","1,1500","4,2700","12,9600","0,50","7.792.811.693,25","96,64",BRKNIPCTF001,30/11/2022,"0,5",NOVEMBRO 2022/2022,13/12/2022,"12,96"
1,KNCR11,"1,1000","3,4000","7,0500","12,8000","1,10","5.594.359.606,83","100,68",BRKNCRCTF000,30/11/2022,"1,1",NOVEMBRO 2022/2022,13/12/2022,"12,42"
2,HGLG11,"1,1000","3,3000","8,8000","16,0500","1,10","3.454.272.241,76","147,26",BRHGLGCTF004,30/11/2022,"1,1",Novembro/2022,14/12/2022,"16,05"
3,IRDM11,"0,7016","2,3400","6,2191","13,7443","0,70","3.450.887.726,43","94,72",BRIRDMCTF004,09/12/2022,"0,701563187",Novembro-2022/2022,16/12/2022,"13,744345626"
4,KNRI11,"0,9100","2,7300","5,3900","10,3800","0,91","3.871.638.686,32","160,27",BRKNRICTF007,30/11/2022,"0,91",NOVEMBRO 2022/2022,14/12/2022,"10,38"
5,CPTS11,"0,3700","2,2700","5,5700","12,1700","0,37","2.911.469.296,79","91,61",BRCPTSCTF004,12/12/2022,"0,37",Novembro-2022/2022,19/12/2022,"12,17"
6,XPLG11,"0,7400","2,2200","4,3600","8,3000","0,74","3.102.791.995,75","114,49",BRXPLGCTF002,30/11/2022,"0,74",11-2022/2022,14/12/2022,"8,3"
7,HCTR11,"1,1000","3,3000","7,1500","16,6700","1,10","2.681.663.950,61","121,43",BRHCTRCTF003,07/12/2022,"1,1",11-2022/2022,14/12/2022,"16,67"
8,RECR11,"0,7350","1,7773","4,7478","13,0406","0,74","2.520.730.032,03","95,33",BRRECRCTF004,07/12/2022,"0,735",NOVEMBRO/2022/2022,14/12/2022,"13,0406"
9,MXRF11,"0,0800","0,2800","0,6100","1,2000","0,08","2.305.298.861,59","10,20",BRMXRFCTF008,30/11/2022,"0,08",Novembro-2022/2022,14/12/2022,"1,2"


In [ ]:
consolidado.to_excel(r'/fiiscrapping.xlsx', sheet_name='Sheet1', index = False)
#!jupyter nbconvert --to html /01_FII_Scrapping_Fundsexplorer.ipynb